<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Warto%C5%9Bci_odstaj%C4%85ce/colabs/Namierzanie_outlayersow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Potrzebne rzeczy do wczytania datasetu

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1610, done.
remote: Counting objects: 100% (389/389), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 1610 (delta 286), reused 306 (delta 223), pack-reused 1221
Receiving objects: 100% (1610/1610), 6.02 MiB | 12.69 MiB/s, done.
Resolving deltas: 100% (972/972), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
!git checkout Wartości_odstające

Branch 'Wartości_odstające' set up to track remote branch 'Wartości_odstające' from 'origin'.
Switched to a new branch 'Wartości_odstające'


In [4]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 24.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=1dadd0241feeed46d1e251db213721799c4fdddd5bed55f342f20540ecbc7e24
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2023-01-17 13:51:59--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 

In [5]:
from lib.pyspark_init import create_spark_context, add_kaggle_data
from lib.const import JOINED_DATA, DATA_WITH_AWARDS

In [6]:
spark = create_spark_context()
spark

In [7]:
%%time
data = spark.read.parquet(JOINED_DATA)
data.show(60)

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|         primaryName|      knownForTitles|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+
|nm0000034|[tt0032636, tt003...|[movie, movie, mo...|[Invisible Stripe...|[0, 0, 0, 0, 0, 0...|     1939|     \N|[Action, Crime, D...|   actor|[["Tim Taylor"], ...|      William Holden|tt0043014,tt00463...|
|nm0000068|[tt0038613, tt002...|[movie, movie, mo...|[Home, Sweet Homi...|[0, 0, 0, 0, 0, 0...|     1931|     \N|[Adventure, Comed...|   actor|[["Lt. Bill Smith...|      Ra

In [8]:
data.createOrReplaceTempView("data");

#Sprawdzanie duplikatow w kolumnach 'nconst' oraz 'primaryName':

In [9]:
spark.sql("""select COUNT( DISTINCT nconst)as IDs, COUNT(nconst) as l_aktorow
FROM data""").show()

+-------+---------+
|    IDs|l_aktorow|
+-------+---------+
|2013276|  2013276|
+-------+---------+



Wniosek - nie ma duplikatów ID aktorow.

To samo dla pozostałych kolumn:

In [10]:
spark.sql("""select COUNT( DISTINCT category) as category, COUNT( DISTINCT primaryName) as primaryName, COUNT(primaryName) as l_wystapien
FROM data""").show()

+--------+-----------+-----------+
|category|primaryName|l_wystapien|
+--------+-----------+-----------+
|       2|    1806848|    2013276|
+--------+-----------+-----------+



In [11]:
spark.sql("""SELECT primaryName, COUNT(*) as l_wystapien
FROM data
GROUP BY primaryName
HAVING COUNT(*) > 1
ORDER BY l_wystapien DESC""").show(150)

+---------------+-----------+
|    primaryName|l_wystapien|
+---------------+-----------+
|           Alex|        120|
|           Anna|        113|
|          Angel|         95|
|        Sabrina|         84|
|          Maria|         84|
|         Nicole|         80|
|        Jessica|         78|
|       Michelle|         78|
|         Monica|         77|
|            Eva|         77|
|         Sandra|         73|
|          Jenny|         72|
|           Mike|         72|
|        Vanessa|         71|
|          David|         69|
|          Chris|         68|
|           Lisa|         67|
|        Jasmine|         66|
|        Claudia|         65|
|          Amber|         64|
|          Julia|         63|
|         Andrea|         62|
|         Daniel|         61|
|           Gina|         61|
|         Carmen|         60|
|          James|         60|
|           Lola|         60|
|          Nikki|         59|
|       Samantha|         58|
|        Charlie|         58|
|         

Widzimy dużo powtórzeń. Wyciągnijmy jak wygladają wiersze z primaryName = 'Alex'

In [12]:
spark.sql("""SELECT * from data where primaryName = 'Alex'""").show(50)

+----------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+-----------+--------------------+
|    nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|primaryName|      knownForTitles|
+----------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+-----------+--------------------+
|nm11906448|[tt15214950, tt14...|[tvEpisode, tvEpi...|[If Ores Grew On ...|           [0, 0, 0]|     2014|     \N|[Action, Adventur...|   actor|[["Futurian Robot...|       Alex|          tt14187980|
| nm8820071|         [tt6590612]|         [tvEpisode]|[Clowns ohne Gren...|                 [0]|     2015|     \N|            [Comedy]|   actor|[["Clown (Clowns ...|       Alex|           tt3137534|
| nm8

In [54]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, CountVectorizer
from pyspark.ml.stat import Summarizer
from pyspark.sql.functions import split, col
from pyspark.sql import DataFrame, SparkSession
from typing import List, Literal
import pandas as pd

#Spróbujemy zliczyć ile jest różnych typów filmów i spróbujemy zobaczyć czy są takie np. występujące bardzo mało razy

In [51]:
def count_vectorizer(
    df: DataFrame,
    column: str,
    drop_cols: bool = True
) -> DataFrame:
    cv = CountVectorizer(inputCol=column, outputCol=column + "_vec").fit(df)
    result = (
        cv
        
        .transform(df)

    )
    # print(cv.explainParams())

    if drop_cols:
        result = result.drop(column)

    return result, cv

Zliczymy liczbe roznych wartosci w kolumnie 'titleType'

In [43]:
test, cv = count_vectorizer(data, 'titleType', False)
test.show(10)

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+------------------+--------------------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|       primaryName|      knownForTitles|       titleType_vec|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+------------------+--------------------+--------------------+
|nm0000034|[tt0032636, tt003...|[movie, movie, mo...|[Invisible Stripe...|[0, 0, 0, 0, 0, 0...|     1939|     \N|[Action, Crime, D...|   actor|[["Tim Taylor"], ...|    William Holden|tt0043014,tt00463...|(11,[1,2,5],[1.0,...|
|nm0000068|[tt0038613, tt002...|[movie, movie, mo...|[Home, Sweet Homi...|[0, 0, 0, 0, 0, 0...| 

Takie sa rozne wartosci w kolumnie 'titleType'

In [44]:
cv.vocabulary

['tvEpisode',
 'short',
 'movie',
 'tvSeries',
 'video',
 'tvMovie',
 'tvMiniSeries',
 'videoGame',
 'tvSpecial',
 'tvShort',
 'tvPilot']

In [45]:
test.agg(Summarizer.sum(test.titleType_vec)).collect()

[Row(sum(titleType_vec)=DenseVector([13952207.0, 1886196.0, 1836075.0, 815491.0, 808665.0, 379648.0, 182005.0, 53797.0, 27172.0, 17391.0, 1.0]))]

Natomiast taka jest licznosc wystapien danej wartosci
na przyklad tvEpisode wystepuje 13952207 razy, natomiast tvPilot tylko 1 raz

Zrobimy tak samo z kolumna genres

In [47]:
test2, cv2 = count_vectorizer(data, 'genres', False)
test2.show(10)

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+------------------+--------------------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|       primaryName|      knownForTitles|          genres_vec|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+------------------+--------------------+--------------------+
|nm0000034|[tt0032636, tt003...|[movie, movie, mo...|[Invisible Stripe...|[0, 0, 0, 0, 0, 0...|     1939|     \N|[Action, Crime, D...|   actor|[["Tim Taylor"], ...|    William Holden|tt0043014,tt00463...|(30,[0,1,2,3,4,5,...|
|nm0000068|[tt0038613, tt002...|[movie, movie, mo...|[Home, Sweet Homi...|[0, 0, 0, 0, 0, 0...| 

Wystepujace gatunki filmow:

In [49]:
cv2.vocabulary

['Drama',
 'Comedy',
 'Romance',
 'Short',
 'Family',
 'Action',
 'Crime',
 'Animation',
 'Adventure',
 'Adult',
 '\\N',
 'Mystery',
 'Fantasy',
 'Thriller',
 'Horror',
 'Music',
 'Sci-Fi',
 'History',
 'Sport',
 'Musical',
 'Documentary',
 'Reality-TV',
 'Talk-Show',
 'Biography',
 'Game-Show',
 'Western',
 'War',
 'News',
 'Film-Noir',
 'Experimental']

In [50]:
test2.agg(Summarizer.sum(test2.genres_vec)).collect()

[Row(sum(genres_vec)=DenseVector([10032628.0, 5428396.0, 3574737.0, 2010200.0, 1637228.0, 1389743.0, 1357986.0, 1235449.0, 1218350.0, 896974.0, 783927.0, 664492.0, 625766.0, 545531.0, 473951.0, 331356.0, 323977.0, 287105.0, 208100.0, 204688.0, 200566.0, 172832.0, 164502.0, 155775.0, 128129.0, 112340.0, 99353.0, 75490.0, 3539.0, 2.0]))]

Analogicznie odczytujemy, ze np. gatunek 'Film-Noir' wystapil 3539 razy, natomiast \\N byl przypisany 783927 razy

Pytanie: co robimy z najrzadziej wystepujacymi wartosciami i od jakiej liczby wystapien?